In [ ]:
import at
import sys
from at import Param, ParamArray
from importlib.resources import files, as_file
from timeit import timeit

In [ ]:
fname = 'hmba.mat'
with as_file(files('machine_data') / fname) as path:
    ring = at.load_lattice(path)

# Parameters

Parameters are objects of class {py:class}`.Param` which can be used instead of numeric values as {py:class}`.Element` attributes.

Parameters are initialised with a **scalar** numeric value. They have an optional name, used when printtig the parameter. Optionally they can be bounded.

In [ ]:
total_length = Param(2.5, name="total_length")
dlength = Param(1.0, name="dlength", bounds=(0.0, 5.0))
print(f"{total_length}: {total_length!r}")
print(f"{dlength}: {dlength!r}")

The value of a parameter can be read or modified through its {py:attr}`~.Param.value` property. {py:meth}`~.VariableBase.set` and {py:meth}`~.VariableBase.get` methods are also available:

In [ ]:
total_length.value = 2.4
print(f"{total_length}: {total_length!r}")

In [ ]:
total_length.set(2.3)
print(total_length.get())

Arithmetic combinations of parameters create new read-only parameters of class {py:class}`.ParamBase`, whose value is permanently kept up-to-date:

In [ ]:
qlength = total_length - dlength
print(f"{qlength}: {qlength!r}")

In [ ]:
dlength.value = 0.9
print(f"{qlength}: {qlength!r}")

Parameters may be assigned to {py:class}`.Element` attributes, for instance on initialisation:

In [ ]:
dr1 = at.Drift('DR1', dlength)
qf1 = at.Quadrupole('QF1', qlength, 0.6)
print(dr1)
print(qf1)

The {py:class}`.Element` attributes keep their type so that all the processing of elements either in python functions or in C integrators is unchanged:

In [ ]:
print(dr1.Length, type(dr1.Length))

## Assigning parameters

### To a single element
Parameters may be assigned to {py:class}`.Element` attributes in several ways:

**At element creation:**

In [ ]:
dr2 = at.Drift('DR2', dlength)

**By converting a numeric attribute into a parameter:**

In [ ]:
qd1 = at.Quadrupole('QD1', 0.5, -0.4)
ql = qd1.parameterise('Length')
vkick = qd1.parameterise("PolynomA", index=0, name="vkick")
print(f"{ql}: {ql!r}")

**By normal assignment, only for scalar parameters:**

In [ ]:
qd1.Length = Param(0.2)
print(qd1)

**With the {py:meth}`~.Element.set_parameter` method:**

In [ ]:
qstrength = Param(-0.5, name='quad_strength')
qd1.set_parameter('PolynomB', qstrength, index=1)
print(qd1)

### To selected elements of a {py:class}`.Lattice`
To act on several elements in a single step, {py:class}`.Lattice` methods similar to {py:class}`.Element` methods are available.

**Convert numeric attributes into parameters:**

The attribute of all the selected elements is replaced by a single parameter whose initial value is the average of the original values.

In [ ]:
kf1 = ring.parameterise('QF1[AE]', 'PolynomB', index=1, name='kf1')
print(f"{kf1}: {kf1!r}")
print(ring[5])

**Use the {py:meth}`~.Lattice.set_parameter` method:**

The attribute of all the selected elements is replaced by the provided parameter,

In [ ]:
lf1 = Param(0.311896, name='lf1')
ring.set_parameter('QF1[AE]', 'Length', lf1)
print(ring[117])

Once a Parameter is assigned to an attribute, it acquires the type and the constraints of
the attribute. For instance:

In [ ]:
num_int_steps = Param(14.4, name="num_int_steps")
ring.set_parameter(at.Multipole, 'NumIntSteps', num_int_steps)
print(ring[5].NumIntSteps, type(ring[5].NumIntSteps))

In [ ]:
num_int_steps.value = -5

The parameter behaves as the attribute.

## Retrieving parameters

Since the values of {py:class}`.Element` attributes keep their original type, they cannot be used to access the underlying parameter. The only way to retrieve it is to use the {py:meth}`~.Element.get_parameter` method. a {py:obj}`TypeError` is raised if the attribute is not
a Parameter.

In [ ]:
ql = qf1.get_parameter('Length')
print(f"{ql}: {ql!r}")
print("ql is qlength:", ql is qlength)

This also works for items in array attributes:

In [ ]:
qs = qd1.get_parameter('PolynomB', index=1)
print(f"{qs}: {qs!r}")
print("qs is qstrength:", qs is qstrength)

## Checking parametrisation

The {py:meth}`~.Element.is_parameterised` method may be applied to:
- a full element: it returns true is any of its attributes is a parameter,
- an array attribute: it returns true if any of its items is a parameter,
- a scalar attribute or an item of an array.

In [ ]:
print(qd1.is_parameterised())
print(qd1.is_parameterised('PolynomB', index=1))
print(qd1.is_parameterised('PolynomB', index=0))

## Removing parameters
Removing the parameters will "freeze" the element at its current value. The {py:meth}`~.Element.unparameterise` method is defined for both {py:class}`.Element` and {py:class}`.Lattice`, and may be applied to:
- a full element: all the parameters are replaced by their value,
- an array attribute: the whole {py:class}`.ParamArray` is replaced by a numpy array,
- a scalar attribute or an item of an array.

### In an Element

In [ ]:
print(qd1)
qd1.unparameterise('Length')
print(qd1)

In [ ]:
qd1.unparameterise('PolynomB', 1)
print(qd1)

### In a Lattice

In [ ]:
print(ring[5])
ring.unparameterise('QF1[AE]', 'PolynomB', index=1)
print(ring[5])

In [ ]:
ring.unparameterise('QF1[AE]')
print(ring[117])

## Parameter history
Parameter values are kept in an {py:attr}`~.VariableBase.history` buffer. The properties {py:attr}`~.VariableBase.initial_value`, {py:attr}`~.VariableBase.last_value` and {py:attr}`~.VariableBase.previous_value` are also available:

In [ ]:
dlength.value = 1.1
print(dlength.history)
print(dlength.initial_value)
print(dlength.previous_value)

After varying parameters, in matching for instance, the current status can be printed:

In [ ]:
print(dlength.status())

Parameters may be reset to a previous history value with the {py:meth}`~.Param.reset` and {py:meth}`~.variables.Variable.set_previous` methods. The history is shortened accordingly.

In [ ]:
dlength.set_previous()
print(dlength, dlength.history)
dlength.reset()
print(dlength, dlength.history)

## Copying and Saving

Parameters cannot be copied. A shallow or deep copy of a parameter returns the parameter itself.

So in a deep copy of an {py:class}`.Element`, the parameters are preserved.

When saving a lattice with parameterised elements, as a `.mat`, a `.m` or `.repr` file, all parametrisation is removed, and a "frozen" state of the lattice is saved.

In pickle dumps of an {py:class}`.Element`, parameters are preserved.

## Timing tests

In [ ]:
a=timeit("b=qd1.Length", globals=globals(), number=10000)
print(f"Scalar attribute, non-parameterised {a:.4}")
b=timeit("b=dr2.Length", globals=globals(), number=10000)
print(f"Scalar attribute,     parameterised {b:.4} (x{b/a:.3})")
a=timeit("b=qd1.PolynomB[1]", globals=globals(), number=10000)
print(f"Array attribute,  non-parameterised {a:.4}")
b=timeit("b=qd1.PolynomA[0]", globals=globals(), number=10000)
print(f"Array attribute,      parameterised {b:.4} (x{b/a:.3})")
c=timeit("b=qd1.PolynomA[1]", globals=globals(), number=10000)
print(f"Array attribute,  non-parameterised {c:.4} (x{c/a:.3}) in a parameter array")